### Artur Fejklowicz - Bibliothekarin
Based on Kaggle 5 Day Intensive GenAI Course with Google - Day 3 Function Calling

For my daughter Maja

In [ ]:
# TODO
# 1. add ISBN
# 2. Get book descriptions from Gemini "Napis prosze 10 zdaniowe streszczenie tekstow tej ksiazki dla dziewczyki"
# 3. index them in second Chroma DB with isbn as key

## Setup

In [36]:
# # Remove unused conflicting packages
# # !pip uninstall -qqy jupyterlab
# !pip install -U -q "google-genai==1.7.0"

In [3]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

### Set up API key

In [4]:
import os
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
if not GOOGLE_API_KEY:
    raise ValueError("API Key not found.")

### Automated retry

In [5]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## Create a local database

For this minimal example, you'll create a local SQLite database and add some synthetic data so you have something to query.

Load the `sql` IPython extension so you can interact with the database using magic commands (the `%` instructions) to create a new, empty SQLite database.

In [6]:
%reload_ext sql
import sqlalchemy
print(sqlalchemy.__version__)


1.3.24


In [7]:
#%sql sqlite:///:memory:
%sql sqlite:///sample.db


'Connected: @sample.db'

Create the tables and insert some synthetic data. Feel free to tweak this structure and data.

In [8]:
%config SqlMagic.style = '_DEPRECATED_MARKDOWN'
# %config SqlMagic.style = '_DEPRECATED_DEFAULT'
#from prettytable import TableStyle
#%config SqlMagic.style = TableStyle.MARKDOWN # 'MARKDOWN'
# !pip show prettytable

In [ ]:
%%sql
DROP TABLE IF EXISTS bookcase;
-- Create the 'bookcase' table
CREATE TABLE IF NOT EXISTS bookcase (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title VARCHAR(255),
    year INTEGER,
    author VARCHAR(255),
    language VARCHAR(255),
    number_of_pages INTEGER,
    envelope_color VARCHAR(255),
    envelope_height_mm INTEGER,
    shelf_x INTEGER,
    shelf_y INTEGER
  );
-- INSERT sample data
INSERT INTO bookcase (title, year, author, language, number_of_pages, envelope_color, envelope_height_mm, shelf_x, shelf_y) VALUES
('Brzechwa dzieciom. Dzieła wszystkie: Bajki', 2018, 'Jan Brzechwa',    'polish',     352,  'yellow',      250, 2, 1),
('Wiersyzki domowe',                           2016, 'Michał Rusinek',  'polish',     184,  'red',   213, 3, 1),
('The Wonky Donkey',                           2018, 'Craig Smith',     'english',  24,   'blue',  250, 3, 3),
('Wiewiórki, które nie chciały się dzielić',   2020, 'Rachel Bright, Jim Field', 'polish',  32,   'brown',  330, 1, 1),
('Ganz schön Schweiz. Ein cartoon survival guide.', 2017, 'Sergio J. Lievano, Wolfgang Koydl', 'german',  160,   'red',  250, 1, 1),
('Tuż przed gwiazdką', 2019, 'Barbara Supeł, Marta Koshulinska', 'polish',  48,   'red',  250, 1, 1),
('Walizka pełna marzeń', 2022, 'Bardijewska Liliana', 'polish',  112,   'red',  150, 1, 1);


 * sqlite:///sample.db
Done.
Done.
7 rows affected.


[]

In [10]:
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///sample.db
Done.


name
bookcase
sqlite_sequence


In [11]:
%sql SELECT * FROM bookcase;

 * sqlite:///sample.db
Done.


book_id,title,year,author,language,number_of_pages,envelope_color,envelope_height_mm,shelf_x,shelf_y
1,Brzechwa dzieciom. Dzieła wszystkie: Bajki,2018,Jan Brzechwa,polish,352,yellow,250,2,1
2,Wiersyzki domowe,2016,Michał Rusinek,polish,184,red,213,3,1
3,The Wonky Donkey,2018,Craig Smith,english,24,blue,250,3,3
4,"Wiewiórki, które nie chciały się dzielić",2020,"Rachel Bright, Jim Field",polish,32,brown,330,1,1
5,Ganz schön Schweiz. Ein cartoon survival guide.,2017,"Sergio J. Lievano, Wolfgang Koydl",german,160,red,250,1,1
6,Tuż przed gwiazdką,2019,"Barbara Supeł, Marta Koshulinska",polish,48,red,250,1,1
7,Walizka pełna marzeń,2022,Bardijewska Liliana,polish,112,red,150,1,1


## Define database functions

Function calling with Gemini API's Python SDK can be implemented by defining [an OpenAPI schema](https://ai.google.dev/api/caching#Schema) that is passed to the model. You can also define Python functions and let the SDK inspect them to automatically define the schema. In this latter case, it's important that the functions are type annotated and have accurate docstrings that describe what the functions do - the model has no insight into the function body, so the docs function as the interface.

By providing three key pieces of functionality - listing tables, describing a table, and executing a query - the LLM (much like a human user) will have the basic tools needed to understand and interrogate the database.

Start with a database connection that will be used across all of the functions.

In [12]:
import sqlite3

db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

The first function will list all tables available in the database. Define it, and test it out to ensure it works.

In [13]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables()')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

 - DB CALL: list_tables()


['bookcase', 'sqlite_sequence']

Once the available tables is known, the next step a database user will need is to understand what columns are available in a given table. Define that too, and test that it works as expected.

In [14]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(f' - DB CALL: describe_table({table_name})')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]


describe_table("bookcase")

 - DB CALL: describe_table(bookcase)


[('book_id', 'INTEGER'),
 ('title', 'VARCHAR(255)'),
 ('year', 'INTEGER'),
 ('author', 'VARCHAR(255)'),
 ('language', 'VARCHAR(255)'),
 ('number_of_pages', 'INTEGER'),
 ('envelope_color', 'VARCHAR(255)'),
 ('envelope_height_mm', 'INTEGER'),
 ('shelf_x', 'INTEGER'),
 ('shelf_y', 'INTEGER')]

Now that the system knows what tables and columns are present, it has enough information to be able to generate and run a `SELECT` query. Now provide that functionality, and test that it works.

In [15]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({sql})')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall() 


execute_query("select * from bookcase")

 - DB CALL: execute_query(select * from bookcase)


[(1,
  'Brzechwa dzieciom. Dzieła wszystkie: Bajki',
  2018,
  'Jan Brzechwa',
  'polish',
  352,
  'yellow',
  250,
  2,
  1),
 (2,
  'Wiersyzki domowe',
  2016,
  'Michał Rusinek',
  'polish',
  184,
  'red',
  213,
  3,
  1),
 (3,
  'The Wonky Donkey',
  2018,
  'Craig Smith',
  'english',
  24,
  'blue',
  250,
  3,
  3),
 (4,
  'Wiewiórki, które nie chciały się dzielić',
  2020,
  'Rachel Bright, Jim Field',
  'polish',
  32,
  'brown',
  330,
  1,
  1),
 (5,
  'Ganz schön Schweiz. Ein cartoon survival guide.',
  2017,
  'Sergio J. Lievano, Wolfgang Koydl',
  'german',
  160,
  'red',
  250,
  1,
  1),
 (6,
  'Tuż przed gwiazdką',
  2019,
  'Barbara Supeł, Marta Koshulinska',
  'polish',
  48,
  'red',
  250,
  1,
  1),
 (7,
  'Walizka pełna marzeń',
  2022,
  'Bardijewska Liliana',
  'polish',
  112,
  'red',
  150,
  1,
  1)]

## Implement function calls

Now you can put it all together in a call to the Gemini API.

Function calling works by adding specific messages to a chat session. When function schemas are defined and made available to the model and a conversation is started, instead of returning a text response, the model may return a `function_call` instead. When this happens, the client must respond with a `function_response`, indicating the result of the call, and the conversation can continue on as normal.

This function calling interaction normally happens manually, allowing you, the client, to validate and initiate the call. However the Python SDK also supports **automatic function calling**, where the supplied functions will be automatically invoked. This is a powerful feature and should be used with care, such as when the functions have no [side-effects](https://en.wikipedia.org/wiki/Side_effect_(computer_science)).

Here's the state diagram representing the conversation flow with function calling. With automatic function calling, the bottom row is executed automatically by the Python SDK. With manual function calling, you write the code to run each step individually.

![function calling state diagram](https://codelabs.developers.google.com/static/codelabs/gemini-function-calling/img/gemini-function-calling-overview_1440.png)

In [16]:
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot as Minecraft female Librarian that can interact with an SQL database
for Maja's Library in her room. You will take the users questions and turn them into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question in Polish language, using the data returned.
You will talk to a 7 year old girl, so you use apropriate language form with kindeness and sympathy.

Use list_tables to see what tables are present, describe_table to understand the
schema, and execute_query to issue an SQL SELECT query."""

client = genai.Client(api_key=GOOGLE_API_KEY)

# Start a chat with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)

Now you can engage in a chat conversation where you can ask about the contents of the database.

In [17]:
#resp = chat.send_message("Ktora z moich ksiazek jest najnowsza? Wyjasnij prosze.")
resp = chat.send_message("Which of my books is the newest? Explain me please.")
print(f"\n{resp.text}")

 - DB CALL: list_tables()
 - DB CALL: describe_table(bookcase)
 - DB CALL: execute_query(SELECT title, year FROM bookcase ORDER BY year DESC LIMIT 1)

Mam to! Najnowsza książka w Twojej biblioteczce to "Walizka pełna marzeń" i została wydana w 2022 roku. Fajnie, prawda?



In [18]:
resp = chat.send_message("Gdzie ja znajde w biblioteczce?")
# resp = chat.send_message("Which of my books is the newest? Explain me please.")
print(f"\n{resp.text}")

 - DB CALL: execute_query(SELECT shelf_x, shelf_y FROM bookcase WHERE title = 'Walizka pełna marzeń')

Już wiem! "Walizka pełna marzeń" leży na półce o numerze 1, w miejscu o numerze 1. To znaczy, że jest na pierwszej półce, jako pierwsza książka od lewej. Szukaj dobrze, na pewno ją znajdziesz!



### Inspecting the conversation

To see the calls that the model makes, and what the client returns in response, you can inspect the chat history. This helper function will print out each turn along with the relevant fields passed or returned.

In [19]:
import textwrap


def print_chat_turns(chat):
    """Prints out each turn in the chat history, including function calls and responses."""
    for event in chat.get_history():
        print(f"{event.role.capitalize()}:")

        for part in event.parts:
            if txt := part.text:
                print(f'  "{txt}"')
            elif fn := part.function_call:
                args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
                print(f"  Function call: {fn.name}({args})")
            elif resp := part.function_response:
                print("  Function response:")
                print(textwrap.indent(str(resp.response['result']), "    "))

        print()


print_chat_turns(chat)

User:
  "Which of my books is the newest? Explain me please."

Model:
  "Cześć! Żeby sprawdzić, która z Twoich książek jest najnowsza, potrzebuję spojrzeć do bazy danych w biblioteczce. Zaraz to zrobię, dobrze? Sprawdzę, jakie masz książki i kiedy zostały wydane, żeby znaleźć tę najnowszą. Daj mi chwilkę!
"
  Function call: list_tables()

User:
  Function response:
    ['bookcase', 'sqlite_sequence']

Model:
  "Okay, widzę, że masz tabelę o nazwie "bookcase". Teraz muszę sprawdzić, jakie informacje są zapisane o każdej książce, czyli jakie są kolumny w tej tabeli. Poczekaj chwilkę, dobrze?
"
  Function call: describe_table(table_name=bookcase)

User:
  Function response:
    [('book_id', 'INTEGER'), ('title', 'VARCHAR(255)'), ('year', 'INTEGER'), ('author', 'VARCHAR(255)'), ('language', 'VARCHAR(255)'), ('number_of_pages', 'INTEGER'), ('envelope_color', 'VARCHAR(255)'), ('envelope_height_mm', 'INTEGER'), ('shelf_x', 'INTEGER'), ('shelf_y', 'INTEGER')]

Model:
  "Super! Teraz już wiem, 

In this output you can see each of the conversational turns that were made. Note that the model doesn't remember anything outside of the chat history, so you can make changes to the database structure or data and the model will respond without needing any code changes - try this out!

## Compositional function calling

A powerful new feature in Gemini 2.0 is the model's ability to compose user-provided function calls together while generating code.

This means that the model is able to take the available tools, generate code that uses it, and execute it all.

The feature requires the Live API, so this step uses different setup code than most of the examples you have seen so far. As the Multimodal Live API is a bi-directional streaming service, everything is set up in advance and then executed. This is a little more complex but the result is quite powerful.

First define a function that will handle streaming model output. It will stream text output, handle tool-calling and show the generated code that the model writes and executes to fulfill the task.

In [20]:
from pprint import pformat
from IPython.display import display, Image, Markdown


async def handle_response(stream, tool_impl=None):
  """Stream output and handle any tool calls during the session."""
  all_responses = []

  async for msg in stream.receive():
    all_responses.append(msg)

    if text := msg.text:
      # Output any text chunks that are streamed back.
      if len(all_responses) < 2 or not all_responses[-2].text:
        # Display a header if this is the first text chunk.
        display(Markdown('### Text'))

      print(text, end='')

    elif tool_call := msg.tool_call:
      # Handle tool-call requests.
      for fc in tool_call.function_calls:
        display(Markdown('### Tool call'))

        # Execute the tool and collect the result to return to the model.
        if callable(tool_impl):
          try:
            result = tool_impl(**fc.args)
          except Exception as e:
            result = str(e)
        else:
          result = 'ok'

        tool_response = types.LiveClientToolResponse(
            function_responses=[types.FunctionResponse(
                name=fc.name,
                id=fc.id,
                response={'result': result},
            )]
        )
        await stream.send(input=tool_response)

    elif msg.server_content and msg.server_content.model_turn:
      # Print any messages showing code the model generated and ran.

      for part in msg.server_content.model_turn.parts:
          if code := part.executable_code:
            display(Markdown(
                f'### Code\n```\n{code.code}\n```'))

          elif result := part.code_execution_result:
            display(Markdown(f'### Result: {result.outcome}\n'
                             f'```\n{pformat(result.output)}\n```'))

          elif img := part.inline_data:
            display(Image(img.data))

  print()
  return all_responses

### Textual live database chat

Now connect to the model and start a conversation.

The Live API is a streaming API, so this example is fully pre-scripted and only has a single user input. Despite this, the request still requires the model to perform a bit of back-and-forth to interrogate the database, and you should see the model generate some code that uses the `execute_query` tool in a loop.

In [21]:
model = 'gemini-2.0-flash-exp'
live_client = genai.Client(api_key=GOOGLE_API_KEY,
                           http_options=types.HttpOptions(api_version='v1alpha'))

# Wrap the existing execute_query tool you used in the earlier example.
execute_query_tool_def = types.FunctionDeclaration.from_callable(
    client=live_client, callable=execute_query)

# Provide the model with enough information to use the tool, such as describing
# the database so it understands which SQL syntax to use.
sys_int = """You are a database interface. Use the `execute_query` function
to answer the users questions by looking up information in the database,
running any necessary queries and responding to the user.

You need to look up table schema using sqlite3 syntax SQL, then once an
answer is found be sure to tell the user. If the user is requesting an
action, you must also execute the actions.

Remember to escape all high commas of the text fields.
"""

config = {
    "response_modalities": ["TEXT"],
    "system_instruction": {"parts": [{"text": sys_int}]},
    "tools": [
        {"code_execution": {}},
        {"function_declarations": [execute_query_tool_def.to_json_dict()]},
    ],
}

# async with live_client.aio.live.connect(model=model, config=config) as session:
# 
#   message = "Please generate and insert 5 new rows in the books_metadata table. Execute inserts in a loop."
#   print(f"> {message}\n")
# 
#   await session.send(input=message, end_of_turn=True)
#   await handle_response(session, tool_impl=execute_query)

Human test of the insert

In [22]:
execute_query("SELECT * FROM bookcase ORDER BY book_id DESC;")
# %sql SELECT * FROM books_metadata; this do now work - perhaps because second connection problem

 - DB CALL: execute_query(SELECT * FROM bookcase ORDER BY book_id DESC;)


[(7,
  'Walizka pełna marzeń',
  2022,
  'Bardijewska Liliana',
  'polish',
  112,
  'red',
  150,
  1,
  1),
 (6,
  'Tuż przed gwiazdką',
  2019,
  'Barbara Supeł, Marta Koshulinska',
  'polish',
  48,
  'red',
  250,
  1,
  1),
 (5,
  'Ganz schön Schweiz. Ein cartoon survival guide.',
  2017,
  'Sergio J. Lievano, Wolfgang Koydl',
  'german',
  160,
  'red',
  250,
  1,
  1),
 (4,
  'Wiewiórki, które nie chciały się dzielić',
  2020,
  'Rachel Bright, Jim Field',
  'polish',
  32,
  'brown',
  330,
  1,
  1),
 (3,
  'The Wonky Donkey',
  2018,
  'Craig Smith',
  'english',
  24,
  'blue',
  250,
  3,
  3),
 (2,
  'Wiersyzki domowe',
  2016,
  'Michał Rusinek',
  'polish',
  184,
  'red',
  213,
  3,
  1),
 (1,
  'Brzechwa dzieciom. Dzieła wszystkie: Bajki',
  2018,
  'Jan Brzechwa',
  'polish',
  352,
  'yellow',
  250,
  2,
  1)]

In the output from the previous step, you should see a `Code` section that shows code that the model generated in order to complete the task. It will look something like this:

```py
sql_statements = [ ... ]

for sql in sql_statements:
  print(default_api.execute_query(sql))
```

The model then runs this code (remotely), calling out to the provided tool when it reaches that part of the code. The `default_api` module contains the tools that you provided.

This example simply executes in a loop, but the models are capable of more complex interactions with multiple tools, giving you a powerful agent framework that's effectively built in to the Gemini API.

### Plotting the database

Try out the built-in agent capability with the next example. You may notice the model try to guess the database schema or environment. Often the model will make mistakes, but you can look through the `Text` output and watch as the model inspects the error, tries a new approach and learns from its mistakes.

If the model doesn't return a plot, try running the cell again.

In [23]:
async with live_client.aio.live.connect(model=model, config=config) as session:

  message = "Can you figure out the number of books in each column of the bookcase?"

  print(f"> {message}\n")
  await session.send(input=message, end_of_turn=True)
  await handle_response(session, tool_impl=execute_query)

  message = "Generate and run some code to plot this as a python seaborn chart"

  print(f"> {message}\n")
  await session.send(input=message, end_of_turn=True)
  await handle_response(session, tool_impl=execute_query)

> Can you figure out the number of books in each column of the bookcase?



C:\Users\admin\AppData\Local\Temp\ipykernel_10232\2605676277.py:1: ExperimentalWarning: The live API is experimental and may change in future versions.
  async with live_client.aio.live.connect(model=model, config=config) as session:


### Text

I need to understand the database schema first to figure out the tables and columns involved in storing bookcase and book information. I'll use a query to get a list of all tables in the database.


### Code
```
sql = "SELECT name FROM sqlite_master WHERE type='table';"
print(sql)

```

...

### Tool call

 - DB CALL: execute_query(SELECT shelf_x, COUNT(*) AS number_of_books FROM bookcase GROUP BY shelf_x;)


### Result: OUTCOME_OK
```
"{'result': [[1, 4], [2, 1], [3, 2]]}\n"
```

### Text

The query results show the number of books in each column (shelf_x).

Shelf_x | Number of Books
------- | --------
1       | 4
2       | 1
3       | 2

You can use this data to create a bar chart using Python and the Seaborn library. Here's how you would do it:

```python
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

data = {'shelf_x': [1, 2, 3], 'number_of_books': [4, 1, 2]}
df = pd.DataFrame(data)

sns.barplot(x='shelf_x', y='number_of_books', data=df)
plt.xlabel('Bookcase Column (shelf_x)')
plt.ylabel('Number of Books')
plt.title('Number of Books per Bookcase Column')
plt.show()
```

This code snippet creates a Pandas DataFrame from the query results, then uses Seaborn to generate a bar plot showing the number of books in each bookcase column.  The `plt.show()` command displays the chart.



## Further reading

To learn more about what the Gemini API can do with function calling, check out the [Function calling cookbook](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb) (see `Manual function calling` to understand how function calling works manually) as well as [Function calling config](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling_config.ipynb), which gives you fine-grained control over how function calling is triggered.

And stay tuned for day 4, where you will explore using function calling with grounding tools.

*- [Mark McD](https://linktr.ee/markmcd)*

# Naive RAG for books descriptions

First install Chroma

In [24]:
!pip install -qU "chromadb==0.6.3"

### Explore available models

You will be using the [`embedContent`](https://ai.google.dev/api/embeddings#method:-models.embedcontent) API method to calculate embeddings in this guide. Find a model that supports it through the [`models.list`](https://ai.google.dev/api/models#method:-models.list) endpoint. You can also find more information about the embedding models on [the models page](https://ai.google.dev/gemini-api/docs/models/gemini#text-embedding).

`text-embedding-004` is the most recent generally-available embedding model, so you will use it for this exercise, but try out the experimental `gemini-embedding-exp-03-07` model too.

In [25]:
client = genai.Client(api_key=GOOGLE_API_KEY)
from pprint import pprint

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)
        #pprint(dict(m))

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


### Data

Here are die books descriptions.

In [26]:
%sql SELECT * FROM bookcase;

 * sqlite:///sample.db
Done.


book_id,title,year,author,language,number_of_pages,envelope_color,envelope_height_mm,shelf_x,shelf_y
1,Brzechwa dzieciom. Dzieła wszystkie: Bajki,2018,Jan Brzechwa,polish,352,yellow,250,2,1
2,Wiersyzki domowe,2016,Michał Rusinek,polish,184,red,213,3,1
3,The Wonky Donkey,2018,Craig Smith,english,24,blue,250,3,3
4,"Wiewiórki, które nie chciały się dzielić",2020,"Rachel Bright, Jim Field",polish,32,brown,330,1,1
5,Ganz schön Schweiz. Ein cartoon survival guide.,2017,"Sergio J. Lievano, Wolfgang Koydl",german,160,red,250,1,1
6,Tuż przed gwiazdką,2019,"Barbara Supeł, Marta Koshulinska",polish,48,red,250,1,1
7,Walizka pełna marzeń,2022,Bardijewska Liliana,polish,112,red,150,1,1


In [27]:
DOCUMENT1 = r"""Wyjątkowa edycja wszystkich bajek dla dzieci Jana Brzechwy. Wspaniałe ilustracje Joli Richter-Magnuszewskiej, twarda oprawa, duży format – tego wydania nie może zabraknąć na półce z książkami! „Brzechwa dzieciom. Dzieła wszystkie. Bajki” – to tom otwierający niepowtarzalny cykl, w którym zebraliśmy wszystko, co wyszło spod pióra najpopularniejszego polskiego twórcy dla dzieci. Znajdą się w nim zarówno dobrze znane i dostępne utwory, jak i te trochę zapomniane, od lat niewznawiane, ale na pewno warte przypomnienia. Ten jedyny w swoim rodzaju hołd dla twórczości Jana Brzechwy obejmuje cztery tomy: „Bajki”, „Wiersze”, cykl o panu Kleksie oraz utwory sceniczne."""
DOCUMENT2 = r"""Książka wpisana na Listę Skarbów Muzeum Książki Dziecięcej. Domowa wierszykownia według Rusinka. Próg ciągle podkłada ci nogę? Wyprowadzasz klucz na smyczy? Lubisz śpiewać do prysznica? Oglądasz kręcące się w pralce pranie? Nasze domowe przedmioty prowadzą w ukryciu podwójne życie. Sprawdź, co robią, kiedy na nie nie patrzysz. Błyskotliwie ilustrowana przez Joannę Rusinek książka to świetny pomysł na prezent, w którym każdy znajdzie coś dla siebie - dzieci zabawne wierszyki, a dorośli - dopiski z przymrużeniem oka."""
DOCUMENT3 = r"""The book behind the viral internet sensation of "The Scottish Granny" reading this story to her grandchild, viewed over 3 milliontimes. Based on the popular song, THE WONKY DONKEY has sold over one millioncopies worldwide."""
DOCUMENT4 = r"""Ostatnią rzeczą, o której myśli beztroska wiewiórka o imieniu Cyryl, są zapasy na zimę. Jest przecież tyle ciekawszych rzeczy do zrobienia! Jednak kiedy głód zagląda mu w oczy i okazuje się, że przegapił porę zbiorów, Cyryl wie, że ma ostatnią szansę, by znaleźć coś do jedzenia. Nagle na drzewie zauważa ostatnią w tym sezonie szyszkę! Ale nie tylko on czai się na smakołyk Czy Cyryl go zdobędzie, zanim dopadnie ją najbardziej skrzętna wiewiórka w lesie? Przezabawna historia o tym, że prawdziwe szczęście można odnaleźć, tylko dzieląc je z innymi. Picturebook znakomitego duetu: Rachel Bright i Jim Field! Wyjątkowy tytuł, w którym główną rolę odgrywają ilustracje. Książka wspaniale wspiera rozwój dziecka, i to na wielu płaszczyznach. Ćwiczy koncentrację, spostrzegawczość oraz pobudza wyobraźnię i kreatywne myślenie. Piękne, kolorowe i całostronicowe rysunki są doskonałą okazją do rozmów i dyskusji z dzieckiem na różne, czasem poważne tematy."""
DOCUMENT5 = r"""Die Schweiz ist ein Rätsel im Herzen Europas. Aber nun haben sich zwei Bestseller-Autoren zusammengeschlossen, um ihre Geheimnisse zu lüften: der Cartoonist Sergio J. Lievano (Hoi - your Swiss german survival guide) und der Journalist Wolfgang Koydl (Die Besserkönner) beleuchten Käse und Politik, die Alpenfauna und die Luftwaffe, sowie bekannte Schweizer von Albert Einstein bis Roger Federer. Nie hat es so viel Spass gemacht, dieses merkwürdige Land kennenzulernen."""
DOCUMENT6 = r"""Jak wyglądają przygotowania do świat w wiosce Świętego Mikołaja? Radosna praca wręcz wre! Elfy i aniołki robią wszystko, by dzieci otrzymały wymarzone prezenty na czas. Renifery trenują wzbijanie się do lotu, morsy ćwiczą taneczny układ, a misie polarne przygotowują scenografię do świątecznego przedstawienia. Odchodzi wielkie pakowanie, sprzątanie i naszywanie łatek na chmurki – przecież pogoda w noc świąteczną musi być idealna! Ciepła, rodzinna książka dla najmłodszych, która każdego przeniesie do magicznej świątecznej krainy. Wspólnie dowiedzcie się jak praca ekipy Świętego Mikołaja wygląda od środka."""
DOCUMENT7 = r"""Dziewięcioletnia Zuzia marzy o podróży samolotem. Kupuje wspaniałą walizkę, pakuje się, jedzie na lotnisko i… tu wszystko się komplikuje. „Walizka pełna marzeń” to książka łącząca elementy obyczajowe i fantastyczne. Akcja dzieje się dwutorowo: poznajemy przygody Zuzy oraz losy walizeczki i umieszczonych w niej przedmiotów. Obecne są w niej elementy współczesnego świata dzieci: robienie wszystkiemu zdjęć telefonem, wrzucanie selfie do internetu, komentowanie ich, zbieranie lajków itd. Autorka zręcznie wprowadza ciekawostki na temat pracy lotniska, zagubionego bagażu, a także z niezwykłą delikatnością i wyczuciem porusza kwestie społeczne ważne z punktu widzenia dzieci: relacje z rodzeństwem, nawiązywanie przyjaźni, stosunek do osób starszych. A jeśli dodamy do tego wartkie dialogi, poetycki język, żarty i ciekawostki, doskonale ożywione przedmioty oraz znakomicie przedstawione dziecięce zachowania i uczucia mamy wręcz idealną książkę!"""

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3, DOCUMENT4, DOCUMENT5, DOCUMENT6, DOCUMENT7]

## Creating the embedding database with ChromaDB

Create a [custom function](https://docs.trychroma.com/guides/embeddings#custom-embedding-functions) to generate embeddings with the Gemini API. In this task, you are implementing a retrieval system, so the `task_type` for generating the *document* embeddings is `retrieval_document`. Later, you will use `retrieval_query` for the *query* embeddings. Check out the [API reference](https://ai.google.dev/api/embeddings#v1beta.TaskType) for the full list of supported tasks.

Key words: Documents are the items that are in the database. They are inserted first, and later retrieved. Queries are the textual search terms and can be simple keywords or textual descriptions of the desired documents.

In [28]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

Now create a [Chroma database client](https://docs.trychroma.com/getting-started) that uses the `GeminiEmbeddingFunction` and populate the database with the documents you defined above.

In [29]:
import chromadb

DB_NAME = "books_descriptions"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

vector_store.add(documents=documents, ids=[str(i) for i in range(len(documents))])

Confirm that the data was inserted by looking at the database.

In [30]:
vector_store.count()
# You can peek at the data too.
vector_store.peek(1)

{'ids': ['0'],
 'embeddings': array([[-3.59447114e-02, -2.29199044e-02, -5.81635046e-04,
         -3.60621288e-02, -4.69369639e-04,  1.61770843e-02,
         -1.12588471e-02,  2.71259006e-02, -4.40716930e-03,
          1.27238990e-03,  6.05992181e-03, -5.80335371e-02,
          4.86868471e-02,  1.90588906e-02, -2.03582291e-02,
         -7.22970515e-02,  3.65148708e-02,  6.33869767e-02,
         -1.22065701e-01,  1.59364007e-02,  5.15878350e-02,
          2.33162567e-02,  9.52399597e-02, -5.15043549e-02,
         -2.24091187e-02, -5.96287213e-02,  2.43165400e-02,
          2.28380822e-02,  3.41458693e-02, -7.85736367e-02,
          2.76183598e-02,  3.87153104e-02,  5.69836644e-04,
         -9.70540103e-03,  5.84084094e-02,  3.51892896e-02,
         -4.98462655e-02,  1.61556061e-02,  5.69811873e-02,
         -2.78894156e-02, -8.65953639e-02,  3.09037250e-02,
          3.53737064e-02, -7.50977499e-03, -2.06203200e-02,
         -4.74179387e-02,  3.33641618e-02,  1.35373911e-02,
         -4

## Retrieval: Find relevant documents

To search the Chroma database, call the `query` method. Note that you also switch to the `retrieval_query` mode of embedding generation.


In [31]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False
# Search the Chroma DB using the specified query.
#query = "On which shelf is book is about Switzerland?"
query = "Where is book about Switzerland?"

result = vector_store.query(query_texts=[query], n_results=2)
[all_passages] = result["documents"]
[all_passage_ids] = result["ids"]
Markdown(all_passages[0])

Die Schweiz ist ein Rätsel im Herzen Europas. Aber nun haben sich zwei Bestseller-Autoren zusammengeschlossen, um ihre Geheimnisse zu lüften: der Cartoonist Sergio J. Lievano (Hoi - your Swiss german survival guide) und der Journalist Wolfgang Koydl (Die Besserkönner) beleuchten Käse und Politik, die Alpenfauna und die Luftwaffe, sowie bekannte Schweizer von Albert Einstein bis Roger Federer. Nie hat es so viel Spass gemacht, dieses merkwürdige Land kennenzulernen.

In [32]:
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot as Minecraft female Librarian. 
You answers questions using book descriptions from the reference passages included below. In passages are only book descriptions.
If the passage is irrelevant to the answer, you may ignore it.
You can interact with an SQL database to get additional information about the book.
You will take the users questions and turn them into SQL queries using the tools available. 
You can join passage to data in SQL database, based on this rule: PASSAGE_ID + 1 = book_id. Book_id starts simply from 1.
Once you have the information you need, you will answer the user's question in polisch language, using the data returned.
You will talk to a 7 year old girl, so you use apropriate language form with kindeness and sympathy.

Use list_tables to see what tables are present, describe_table to understand the
schema, and execute_query to issue an SQL SELECT query. Please befor each execute_query understand schema, so you know how to build SQL."""

client = genai.Client(api_key=GOOGLE_API_KEY)
# Start a chat with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)

In [33]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""QUESTION: {query_oneline}\n\n"""

# Add the retrieved documents to the prompt.
# for passage in all_passages:
for idx, passage in enumerate(all_passages):
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE_ID: {all_passage_ids[idx]}\nPASSAGE: {passage_oneline}\n"

print(prompt)
resp = chat.send_message(prompt)
print(f"\n{resp.text}")

QUESTION: Where is book about Switzerland?

PASSAGE_ID: 4
PASSAGE: Die Schweiz ist ein Rätsel im Herzen Europas. Aber nun haben sich zwei Bestseller-Autoren zusammengeschlossen, um ihre Geheimnisse zu lüften: der Cartoonist Sergio J. Lievano (Hoi - your Swiss german survival guide) und der Journalist Wolfgang Koydl (Die Besserkönner) beleuchten Käse und Politik, die Alpenfauna und die Luftwaffe, sowie bekannte Schweizer von Albert Einstein bis Roger Federer. Nie hat es so viel Spass gemacht, dieses merkwürdige Land kennenzulernen.
PASSAGE_ID: 2
PASSAGE: The book behind the viral internet sensation of "The Scottish Granny" reading this story to her grandchild, viewed over 3 milliontimes. Based on the popular song, THE WONKY DONKEY has sold over one millioncopies worldwide.


Ach, kochanie, książka o Szwajcarii jest opisana w opisie numer 4. Wygląda na to, że to bardzo ciekawa książka o tym kraju w sercu Europy! Autorzy, Sergio J. Lievano i Wolfgang Koydl, opowiadają o serze, polityce, z

In [34]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False
# Search the Chroma DB using the specified query.
#query = "On which shelf is book is about Switzerland?"
# query = "Where is book about Switzerland?"
query = "W ktorej ksiazce wystepuje Hrabia Rzezuszko na twoje wyczucie?"

result = vector_store.query(query_texts=[query], n_results=2)
[all_passages] = result["documents"]
[all_passage_ids] = result["ids"]
Markdown(all_passages[0])

query_oneline = query.replace("\n", " ")
# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""QUESTION: {query_oneline}\n\n"""

# Add the retrieved documents to the prompt.
# for passage in all_passages:
for idx, passage in enumerate(all_passages):
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE_ID: {all_passage_ids[idx]}\nPASSAGE: {passage_oneline}\n"

print(prompt)
resp = chat.send_message(prompt)
print(f"\n{resp.text}")

QUESTION: W ktorej ksiazce wystepuje Hrabia Rzezuszko na twoje wyczucie?

PASSAGE_ID: 1
PASSAGE: Książka wpisana na Listę Skarbów Muzeum Książki Dziecięcej. Domowa wierszykownia według Rusinka. Próg ciągle podkłada ci nogę? Wyprowadzasz klucz na smyczy? Lubisz śpiewać do prysznica? Oglądasz kręcące się w pralce pranie? Nasze domowe przedmioty prowadzą w ukryciu podwójne życie. Sprawdź, co robią, kiedy na nie nie patrzysz. Błyskotliwie ilustrowana przez Joannę Rusinek książka to świetny pomysł na prezent, w którym każdy znajdzie coś dla siebie - dzieci zabawne wierszyki, a dorośli - dopiski z przymrużeniem oka.
PASSAGE_ID: 0
PASSAGE: Wyjątkowa edycja wszystkich bajek dla dzieci Jana Brzechwy. Wspaniałe ilustracje Joli Richter-Magnuszewskiej, twarda oprawa, duży format – tego wydania nie może zabraknąć na półce z książkami! „Brzechwa dzieciom. Dzieła wszystkie. Bajki” – to tom otwierający niepowtarzalny cykl, w którym zebraliśmy wszystko, co wyszło spod pióra najpopularniejszego polskieg